In [1]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

model = YOLO('yolov8m.pt')
tracker = DeepSort(max_age=10)

cap = cv2.VideoCapture('SeoulWalk2_720.mp4')
track_paths = {}
color_map = {}

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    results = model(frame, verbose=False)[0]
    #frame = results.plot()

    detections = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))

    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = (x1 + x2) // 2, (y1 + y2) //2
        
        if track_id not in color_map:
            color_map[track_id] = tuple(np.random.randint(0, 255, size=3).tolist())
        color = color_map[track_id]
        
        class_name = track.get_det_class()
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f'ID {track_id} {class_name}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        if track_id not in track_paths:
            track_paths[track_id] = []
        track_paths[track_id].append((cx, cy))
        path = track_paths[track_id]
        for i in range(1, len(path)):
            cv2.line(frame, path[i-1], path[i], color, 2)
    
    cv2.imshow("YOLOv8 plot + Deep Sort Tracking Visualization", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()